In [20]:
import backtrader as bt
import datetime 
import os.path 
import sys

In [21]:
# Create a Data Feed
data = bt.feeds.YahooFinanceCSVData(
    dataname = 'IBM.csv', 
    # Do not pass values before this date
    fromdate=datetime.datetime(2014, 7, 20),
    # Do not pass values after this date
    todate=datetime.datetime(2023, 7, 18),
    reverse=False)


In [67]:
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 20),
        ('printlog', False)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
cerebro = bt.Cerebro()

# Add a strategy
strats = cerebro.optstrategy(
    TestStrategy,
    maperiod=range(10, 31))

cerebro.adddata(data)
cerebro.broker.setcash(1000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=2)
cerebro.broker.setcommission(commission = 0.001)
cerebro.run(maxcpus=1)


In [86]:
class RSITestStrategy(bt.Strategy): 

    params = {'rsi_period': 14, 'holding_period': 200}
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self): 
        self.close = self.datas[0].close
        self.rsi = bt.indicators.RSI(self.datas[0], period = self.params.rsi_period)

        self.order = None

    def notify_order(self, order): 
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self): 

        if not self.position: 
            if self.rsi[0] < 30: 
                # Then we are going to buy
                self.log(f'CLOSE price of {self.close[0]} and RSI at {self.rsi[0]}. Initiating a buy order...')
                self.order = self.buy()

        else: 
            # We are going to exit if the RSI is above 70 or we have held on for too long
            if self.rsi[0] > 70: 
                self.log(f'CLOSE price of {self.close[0]} and RSI at {self.rsi[0]}. We are selling because RSI is > 70') 
                self.order = self.sell()

            elif len(self) > self.bar_executed + self.params.holding_period: 
                self.log(f'CLOSE price of {self.close[0]} and RSI at {self.rsi[0]}. We are selling because we have been in the position for longer than {self.params.holding_period} bars.')
                self.order = self.sell()


        

        

    

In [87]:
c = bt.Cerebro() 
c.addstrategy(RSITestStrategy)
c.adddata(data)
c.broker.setcash(100_000)
print('Starting Portfolio Value: %.2f' % c.broker.getvalue())
c.broker.setcommission(commission = 0.001)
c.run()
print('Final Portfolio Value: %.2f' % c.broker.getvalue())

Starting Portfolio Value: 100000.00
2014-10-16, CLOSE price of 117.59 and RSI at 29.5324413579026. Initiating a buy order...
2014-10-17, BUY EXECUTED, Price: 118.51, Cost: 118.51, Comm 0.12
2015-04-28, CLOSE price of 115.31 and RSI at 70.35171644629683. We are selling because RSI is > 70
2015-04-29, SELL EXECUTED, Price: 114.96, Cost: 118.51, Comm 0.11
2015-08-21, CLOSE price of 100.26 and RSI at 26.68463847915325. Initiating a buy order...
2015-08-24, BUY EXECUTED, Price: 96.63, Cost: 96.63, Comm 0.10
2016-03-14, CLOSE price of 98.06 and RSI at 70.45894027506114. We are selling because RSI is > 70
2016-03-15, SELL EXECUTED, Price: 97.35, Cost: 96.63, Comm 0.10
2017-04-11, CLOSE price of 121.34 and RSI at 29.010900966813125. Initiating a buy order...
2017-04-12, BUY EXECUTED, Price: 121.67, Cost: 121.67, Comm 0.12
2017-10-18, CLOSE price of 115.8 and RSI at 82.70793712201461. We are selling because RSI is > 70
2017-10-19, SELL EXECUTED, Price: 116.00, Cost: 121.67, Comm 0.12
2018-06-25